# Блок №5

# Задание 1
___
Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:

+ InvoiceNo — номер транзакции
+ StockCode — код товара
+ Description — описание товара
+ Quantity — количество единиц товара, добавленных в заказ
+ InvoiceDate — дата транзакции 
+ UnitPrice — цена за единицу товара
+ CustomerID — id клиента
+ Country — страна, где проживает клиент

Данные можно скачать [отсюда](https://disk.yandex.ru/d/q0n4CB1Rf0DxPg).

В данной задаче ссылка для считывания данных лежит в переменной `path_to_file`.
Импортируйте пандас и прочитайте данные с кодировкой `ISO-8859-1`. Запишите полученный датафрейм в `retail`, а названия колонок сохраните в переменную `retail_columns`.

In [1]:
# Импортируем библиотеки
import pandas as pd

In [5]:
# Путь до файла
path = 'https://stepik.org/media/attachments/lesson/361623/data.csv.zip'

In [23]:
# Загружаем данные
retail = pd.read_csv(path,
                    encoding='ISO-8859-1',
                    compression='zip')

In [4]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [7]:
# Сохраним названия колонок в отдельную переменнную
retail_columns = retail.columns

In [7]:
retail_columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

# Задание 2
___
Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из `retail`

In [25]:
# Сохраним первоначальное количество строк в переменную
nrows_original = retail.shape[0]

In [26]:
nrows_original

541909

In [27]:
# Удалим дубликаты и сохраним на месте 
retail.drop_duplicates(inplace=True)

In [28]:
# Сохраним новое количество строк в переменную
nrows = retail.shape[0]

In [29]:
nrows

536641

In [30]:
# Посмотрим были ли дубликаты и их количество
nrows_original - nrows

5268

In [31]:
# Посмотреть количество дубликатов
retail.duplicated().sum()

0

# Задание 3
___
Данные содержат в себе записи как и об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (`InvoiceNo`) ставится `C` (canceled). 

Сколько всего заказов отменили пользователи?

In [32]:
# Посмотрим на формат имеющихся данных
retail.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [33]:
# Обратимся к строковым методам колонки и найдем те, которые начинаются с 'C'
retail.InvoiceNo.str.startswith('C').sum()

9251

# Задание 4
___
Теперь отфильтруйте данные и оставьте в `retail` только те заказы, где` Quantity > 0`. В качестве ответа укажите число оставшихся строк.

In [34]:
# Фильтруем строки и перезаписываем датафрейм
retail = retail \
                .query('Quantity > 0')

In [36]:
# Посмотрим на количество строк
retail.shape[0]

526054

# Задание 5
___
Посчитайте число заказов для каждого пользователя (`CustomerID`) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (`InvoiceNo`), где N – 80% процентиль. Запишите полученные id пользователей в `germany_top` (не весь датафрейм, только id).

Отфильтрованные данные записаны в retail. Идентификатор заказа – InvoiceNo. Для каждого заказа в данных может встречаться более 1 строки.

In [41]:
# Для каждого пользователя из германии, найдем количество транзакций
german_buyers = retail.query('Country == "Germany"') \
    .groupby(['CustomerID']) \
    .agg({'InvoiceNo': pd.Series.nunique}) \
    .rename(columns={'InvoiceNo': 'orders_numbers'})

german_buyers.head()

,orders_numbers
CustomerID,
12426.0,1
12427.0,3
12468.0,2
12471.0,30
12472.0,7


In [42]:
# german_buyers.InvoiceNo.sort_values().iloc[german_buyers.shape[0] // 2]

In [44]:
# Найдем 80 процентиль
percentile_80 = german_buyers.orders_numbers.quantile(q=0.8)

In [45]:
percentile_80

7.0

In [46]:
# Отберем и запишем номера пользователей, количество заказов которых больше 80 процентиля
germany_top = german_buyers.query('orders_numbers > @percentile_80').index

In [47]:
germany_top

Float64Index([12471.0, 12474.0, 12476.0, 12481.0, 12500.0, 12524.0, 12569.0,
              12600.0, 12619.0, 12621.0, 12626.0, 12647.0, 12662.0, 12705.0,
              12708.0, 12709.0, 12712.0, 12720.0],
             dtype='float64', name='CustomerID')

# Задача 6
___
Используя объект с id пользователей (`germany_top`), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в `top_retail_germany`

In [48]:
# Отберем данные по id пользователя и запишем в новый датафрейм
top_retail_germany = retail.query('CustomerID in @germany_top')

In [50]:
top_retail_germany.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany


# Задание 7
___
Сгруппируйте `top_retail_germany` по коду товара (`StockCode`). Какой из продуктов добавляли в корзину чаще всего, кроме `POST`?

In [51]:
# Можно воспользоваться методом value_counts()
top_retail_germany.StockCode.value_counts()

POST      213
22326      52
22328      38
22423      34
20719      30
         ... 
84509E      1
22622       1
22246       1
22660       1
47599A      1
Name: StockCode, Length: 1158, dtype: int64

In [52]:
# Можно воспользовааться группировкой
top_retail_germany \
    .groupby('StockCode') \
    .agg({'Description': 'count'}) \
    .sort_values('Description')

,Description
StockCode,
10125,1
22246,1
23179,1
22242,1
23184,1
...,...
20719,30
22423,34
22328,38


# Задание 8
___
Вернемся к анализу полного датасета `retail`. Создайте колонку `Revenue` с суммой покупки, используя колонки `Quantity` и `UnitPrice`

In [53]:
# Создадим новую колонку
retail['Revenue'] = retail.Quantity * retail.UnitPrice

In [54]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


In [55]:
# retail.drop(columns='Revenue', inplace=True)

# Еще один способ создания колонки
# retail = retail.assign(
#         Revenue = retail.Quantity * retail.UnitPrice
# )

# retail

# Задание 9
___
Для каждой транзакции (`InvoiceNo`), посчитайте финальную сумму заказа. В качестве ответа укажите топ-5 (`InvoiceNo`) по сумме заказа (через запятую с пробелом и в порядке убывания `TotalRevenue`.

Например, для следующего примера...
```
InvoiceNo StockCode Quantity InvoiceDate          UnitPrice  CustomerID  Revenue
536365    85123A    6        2010-12-01 08:26:00  2.55       17850.0     15.30
536365    71053     6        2010-12-01 08:26:00  3.39       17850.0     20.34
555555    71053     1        2010-12-02 18:00:00  3.39       17850.0     3.39
```
...сумма заказа будет равна:
```
InvoiceNo  TotalRevenue
536365     35.64
555555     3.39
```
И ответом будет:
```
536365, 555555
```

In [59]:
# Отсортируем и агрегируем данные
sort_revenues = retail \
                    .groupby(['InvoiceNo'], as_index=False) \
                    .agg({'Revenue': 'sum'}) \
                    .sort_values('Revenue', ascending=False)

In [60]:
# Посмотрим на рещультат
sort_revenues

,InvoiceNo,Revenue
20689,581483,168469.60
2202,541431,77183.60
17582,574941,52940.94
18251,576365,50653.91
9034,556444,38970.00
...,...,...
10074,558724,0.00
10075,558725,0.00
10150,558889,0.00
20726,A563186,-11062.06


In [61]:
# Выпишем первые пять индексов
print(sort_revenues.InvoiceNo[:5].str.cat(sep=', '))

581483, 541431, 574941, 576365, 556444
